In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import pandas as pd

# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 크롬 드라이버 생성
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

# 다이소몰 페이지에 접속
url = "https://www.daisomall.co.kr/pd/pdr/SCR_PDR_0001?pdNo=1049276&recmYn=N"
driver.get(url)
driver.maximize_window()
time.sleep(3)  # 첫 페이지 로딩 대기

# 데이터 저장 리스트
all_reviews = []

# 최대 페이지 수
max_pages = 944
current_page = 1

while current_page <= max_pages:
    try:
        # 현재 페이지의 HTML 소스 가져오기
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 리뷰 블록 선택
        review_blocks = soup.select('li.review-detail')
        
        # 리뷰 데이터 추출
        for block in review_blocks:
            score = block.select_one('span.score')
            star_rating = score.get_text(strip=True) if score else None
            
            review_text_block = block.select_one('div.cont > div')
            review_text = review_text_block.get_text(strip=True) if review_text_block else None
            
            review_info = block.select('ul.info-list.review li')
            moisture_value = None
            stimulus_value = None
            absorption_value = None
            
            for item in review_info:
                item_name = item.select_one('.item').get_text(strip=True)
                item_value = item.select_one('.val').get_text(strip=True)
                
                if '보습력' in item_name:
                    moisture_value = item_value
                elif '자극도' in item_name:
                    stimulus_value = item_value
                elif '흡수력' in item_name:
                    absorption_value = item_value
            
            all_reviews.append({
                'Star Rating': star_rating,
                'Review Text': review_text,
                'Moisture': moisture_value,
                'Stimulus': stimulus_value,
                'Absorption': absorption_value
            })
        
        # 다음 페이지 버튼 클릭
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.btn-next'))
        )
        next_button.click()
        
        # 현재 페이지 번호 증가
        current_page += 1
        time.sleep(3)  # 페이지 로딩 대기
    
    except Exception as e:
        print(f"Error on page {current_page}: {e}")
        break  # 오류 발생 시 종료

# WebDriver 종료
driver.quit()

# 데이터 저장
df = pd.DataFrame(all_reviews)
df.to_csv('VT_RiddleShot_300.csv', index=False, encoding='utf-8-sig')

print("모든 페이지에서 데이터 크롤링 완료! 'daiso_all_reviews33.csv' 파일로 저장되었습니다.")


Error on page 898: Message: 

모든 페이지에서 데이터 크롤링 완료! 'daiso_all_reviews33.csv' 파일로 저장되었습니다.
